# Interactive Mesh Generator Prototype
This prototype uses **gmsh**, **pyvista** and **ipywidgets** to generate and visualize a 3D mesh of random spheres packed inside a cube.

In [ ]:
!pip install gmsh pyvista ipywidgets

In [ ]:
import random
import tempfile
import gmsh
import pyvista as pv
import ipywidgets as widgets
from IPython.display import display, clear_output

In [ ]:
def generate_mesh(radius, box_size, element_size, n_spheres):
    gmsh.initialize()
    gmsh.model.add("pack")
    gmsh.option.setNumber("Mesh.CharacteristicLengthMin", element_size)
    gmsh.option.setNumber("Mesh.CharacteristicLengthMax", element_size)
    gmsh.model.occ.addBox(0, 0, 0, box_size, box_size, box_size, 1)
    sphere_tags = []
    for _ in range(n_spheres):
        x = random.uniform(radius, box_size - radius)
        y = random.uniform(radius, box_size - radius)
        z = random.uniform(radius, box_size - radius)
        tag = gmsh.model.occ.addSphere(x, y, z, radius)
        sphere_tags.append(tag)
    gmsh.model.occ.cut([(3, 1)], [(3, t) for t in sphere_tags], removeObject=False, removeTool=True)
    gmsh.model.occ.synchronize()
    gmsh.model.mesh.generate(3)
    msh_file = tempfile.mktemp(suffix=".msh")
    gmsh.write(msh_file)
    gmsh.finalize()
    return msh_file


def show_mesh(path):
    mesh = pv.read(path)
    plotter = pv.Plotter()
    plotter.add_mesh(mesh, show_edges=True)
    plotter.show(jupyter_backend="pythreejs")

In [ ]:
radius_slider = widgets.FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description='raio')
box_slider = widgets.FloatSlider(value=5.0, min=1.0, max=10.0, step=0.5, description='caixa')
size_slider = widgets.FloatSlider(value=0.5, min=0.1, max=2.0, step=0.1, description='elemento')
num_dropdown = widgets.Dropdown(options=[1, 2, 3, 5, 10], value=3, description='esferas')
output = widgets.Output()

def update(_=None):
    with output:
        clear_output(wait=True)
        path = generate_mesh(radius_slider.value, box_slider.value, size_slider.value, num_dropdown.value)
        show_mesh(path)

for w in (radius_slider, box_slider, size_slider, num_dropdown):
    w.observe(update, names='value')

display(radius_slider, box_slider, size_slider, num_dropdown, output)
update()